In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
from gpiozero import LED

In [14]:
import time

In [3]:
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

In [4]:
def peakloadmode(rawdata,percent_batt):
    batt=percent_batt*96
    Consumption_afterPV=[]
    Charge=[]
    for i in rawdata.index:
        if rawdata.loc[i,'Generation']>=rawdata.loc[i,'Consumption']:
            Charge.append(rawdata.loc[i,'Generation']-rawdata.loc[i,'Consumption'])
            Consumption_afterPV.append(0)
        else:
            Charge.append(0)
            Consumption_afterPV.append(rawdata.loc[i,'Consumption']-rawdata.loc[i,'Generation'])    

    rawdata['Charge']=Charge
    rawdata['Consumption_afterPV']=Consumption_afterPV

    totalcharge=rawdata['Charge'].sum()

    data=rawdata

    peakloadtotal=0

    for i in range(1,96):
        peakloaddf=data.nlargest(i,columns='Consumption_afterPV').Consumption_afterPV
        #print(peakloaddf)
        peakload=peakloaddf.iloc[i-1:i]
        peakloadtotal += peakload.iloc[0]
       # print(peakload)
        #print(peakloadtotal)
        if peakloadtotal>=totalcharge:
            remain=(peakloadtotal-totalcharge)
            #print(remain)
            for j in peakloaddf.index:
                data.loc[j:j,'Consumption_afterPV']=0
            data.loc[peakload.index,'Consumption_afterPV']=remain
            break

    data['Discharge']=data.Consumption_afterPV*0
    for i in peakloaddf.index:
        data.loc[i:i,'Discharge']=data.loc[i:i,'Consumption']-data.loc[i:i,'Generation']-data.loc[i:i,'Consumption_afterPV']

    Contactor=[]
    for i in data.index:
        if data.loc[i,'Charge']>0:
            Contactor.append(1)
        elif data.loc[i,'Discharge']>0:
            Contactor.append(1)
        else:
            Contactor.append(0)
    data['Contactor']=Contactor
    data.rename(columns={'Consumption_afterPV':'Consumption_afterDischarge'},inplace=True)
    data['Consumption_afterPV']=data['Consumption_afterDischarge']+data['Discharge']
    return data

## Create Predict_generation function name predictgen

## PULL data consumption in list (total member = 30*96) name preconsumption

In [ ]:
fit4 = ExponentialSmoothing(preconsumption, seasonal_periods=96, trend='add' ,seasonal='mul').fit()

fit4.fittedvalues.plot(style='--', color='red')

forecast=fit4.forecast(96)

params=fit4.model.params
consumption=[]
for i in forecast:
    consumption.append(i)

## Create function predictcon

In [ ]:
def createDF(predict_date,Consumption,Generation):
    predictday=predict_date[0:4]+predict_date[5:7]+predict_date[8:10]
    minu=[]
    h=[]
    m=[]
    index=[]
    Time=[]
    for i in range(1,96):
        minu.append(str((i%4)*15).zfill(2))
        h.append(str(i//4).zfill(2))
    for i in range(1,96):
        index.append(predictday+h[i-1]+minu[i-1])
    for i in index:
        Time.append(pd.to_datetime(i))
    Time.append(pd.to_datetime(predictday+'0000'))
    len(Time)
    DF = pd.DataFrame()
    DF['Consumption']=Consumption
    DF['Time']=Time
    DF.set_index('Time',inplace=True)
    DF['Generation']=Generation
    return DF

In [ ]:
led = LED(17)

## initial_predict_date must be in the future

In [28]:
initial_predict_date='2019-03-20'

In [30]:
start=0
while start==0:
    if pd.datetime.now()>=pd.to_datetime(initial_predict_date+' 00:00:00'):
        start=1
        predict_date=initial_predict_date
    
    

## PULL PERCENT BATT

In [43]:
stop=0
contactor=[]
while stop==0:
    predict_date=str(pd.datetime.now().date())
    consumption=predictcon(predict_date)
    generation=predictgen(predict_date)
    data=createDF(predict_date,consumption,generation)
    # PULL PERCENT BATT in variable percent_batt
    data=peakloadmode(data,percent_batt)
    for i in data.index:
        while pd.datetime.now()<i:
            if data.loc(i,'Contactor')==1:
                led.on()
            else:
                led.off()
    

NameError: name 'predictcon' is not defined

## TEST